Train MLP for all fingers!\
Using hyper param optimalization
Data source: \
sliding windowed powers for mu and beta band\
Subject 1

In [1]:
import os
import numpy as np
import h5py
from scipy import stats
import scipy.io
import mne

mne.set_log_level('error')

from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold


import torch
import torch.nn as nn
import torch.optim as optim

import optuna


from utils.load import Load
from config.default import cfg

%load_ext autoreload
%autoreload 2


In [2]:
subject_id = 0

In [3]:
device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)
print(device)

cuda


In [4]:
# Load the data  from the HDF5 file
target_dir = 'features'
tag = 'reproduced_with_bad'
file_path = os.path.join(target_dir, tag+'_'+cfg['subjects'][subject_id] + '.h5')


data = {}
with h5py.File(file_path, 'r') as h5file:
    for key in h5file.keys():
        data[key] = np.array(h5file[key])

# Print the loaded data dictionary
for key, value in data.items():
    print(key, value.shape)

index (50, 158, 26, 2)
little (50, 158, 26, 2)
middle (50, 158, 26, 2)
ring (50, 158, 26, 2)
thumb (50, 158, 26, 2)


In [5]:
for key, value in data.items():
    data[key] = value.reshape(value.shape[0], -1)

    
# Print the loaded data dictionary
for key, value in data.items():
    print(f'Value mean {np.mean(value)}, std: {np.std(value)}')
    print(key, value.shape)

Value mean -0.2339724015045144, std: 0.5695447701676682
index (50, 8216)
Value mean -0.2708822967951231, std: 0.5462566462697682
little (50, 8216)
Value mean -0.2442623948850108, std: 0.5700958201096925
middle (50, 8216)
Value mean -0.22332821025493294, std: 0.5757079219489506
ring (50, 8216)
Value mean -0.25530375728577215, std: 0.5697075097118033
thumb (50, 8216)


In [6]:
class SingleLayerMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, activation):
        super(SingleLayerMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.activation = activation
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = nn.Softmax(dim=1)(x)
        return x

In [7]:
data.keys()

dict_keys(['index', 'little', 'middle', 'ring', 'thumb'])

In [8]:
def train(X_train, y_train, X_test, y_test, model, criterion, optimizer, num_epochs=100):
    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.long).to(device)
    y_test = torch.tensor(y_test, dtype=torch.long).to(device)


    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        # # Shuffle X and y together
        p = np.random.permutation(len(X_train))
        X_train, y_train = X_train[p], y_train[p]

    with torch.no_grad():
        y_pred = model(X_test)
        y_pred = torch.argmax(y_pred, dim=1)

    acc = accuracy_score(y_test.to('cpu'), y_pred.to('cpu'))
    return acc

def objective(trial, X, y):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    num_epochs = trial.suggest_int("num_epochs", 100, 2000)
    hidden_size = trial.suggest_int("hidden_size", 16, 128)
    activation_name = trial.suggest_categorical("activation", ["relu", "elu", "leaky_relu"])
    optimizer = trial.suggest_categorical("optimizer", ["SGD", "Adam"])

    if activation_name == "relu":
        activation = nn.ReLU()
    elif activation_name == "elu":
        activation = nn.ELU()
    elif activation_name == "leaky_relu":
        activation = nn.LeakyReLU()

    if optimizer == "SGD":
        optimizer = optim.SGD
    elif optimizer == "Adam":
        optimizer = optim.Adam

    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
    model = SingleLayerMLP(train_X.shape[1], hidden_size, 5, activation)
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(), lr=learning_rate)
    return train(train_X, train_y, test_X, test_y, model, criterion, optimizer, num_epochs=num_epochs)

    # kf = KFold(n_splits=10, shuffle=True, random_state=42)
    # fold_accuracies = []

    # for train_index, test_index in kf.split(X):
    #     X_train, X_test = X[train_index], X[test_index]
    #     y_train, y_test = y[train_index], y[test_index]

    #     model = SingleLayerMLP(X_train.shape[1], hidden_size, 2, activation)
    #     criterion = nn.CrossEntropyLoss()
    #     optimizer = optimizer(model.parameters(), lr=learning_rate)
    #     acc = train(X_train, y_train, X_test, y_test, model, criterion, optimizer, num_epochs=num_epochs)
    #     fold_accuracies.append(acc)

    # mean_accuracy = np.mean(fold_accuracies)
    # return mean_accuracy

def train_MLP(n_trials = 100):
    # Convert data into numpy arrays
    X = np.concatenate(list(data.values()), axis=0)
    y = np.concatenate([np.ones(data[finger].shape[0]) * i for i, finger in enumerate(data)], axis=0)

    # Shuffle X and y together
    # p = np.random.permutation(len(X))
    # X, y = X[p], y[p]

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, X, y), n_trials=n_trials)

    best_trial = study.best_trial

    print(f'Best trial params: {best_trial.params}')
    print(f'Best trial accuracy: {best_trial.value * 100:.2f}%')



In [9]:
train_MLP(n_trials=100)

[I 2023-04-22 18:16:46,173] A new study created in memory with name: no-name-ba171715-05af-41dc-8992-867d65d54e1e
[I 2023-04-22 18:16:54,465] Trial 0 finished with value: 0.38 and parameters: {'learning_rate': 3.679069943290924e-05, 'num_epochs': 790, 'hidden_size': 119, 'activation': 'relu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.38.
[I 2023-04-22 18:17:01,393] Trial 1 finished with value: 0.38 and parameters: {'learning_rate': 6.839071764072327e-05, 'num_epochs': 1371, 'hidden_size': 126, 'activation': 'relu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.38.
[I 2023-04-22 18:17:06,973] Trial 2 finished with value: 0.38 and parameters: {'learning_rate': 1.941415511717045e-05, 'num_epochs': 1321, 'hidden_size': 121, 'activation': 'leaky_relu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.38.
[I 2023-04-22 18:17:12,056] Trial 3 finished with value: 0.34 and parameters: {'learning_rate': 0.04445898426370248, 'num_epochs': 1995, 'hidden_size': 45, 'activation': 

Best trial params: {'learning_rate': 0.0021345711699235683, 'num_epochs': 732, 'hidden_size': 37, 'activation': 'relu', 'optimizer': 'Adam'}
Best trial accuracy: 48.00%


In [10]:
'''
tag = 'gpt4freq_all'
Best trial params: {'learning_rate': 0.009669058999906542, 'num_epochs': 1671, 'hidden_size': 111, 'activation': 'relu', 'optimizer': 'SGD'}
Best trial accuracy: 48.00%

tag = 'reproduced_with_bad'
Best trial params: {'learning_rate': 0.0021345711699235683, 'num_epochs': 732, 'hidden_size': 37, 'activation': 'relu', 'optimizer': 'Adam'}
Best trial accuracy: 48.00%
'''


"\ntag = 'gpt4freq_all'\nBest trial params: {'learning_rate': 0.009669058999906542, 'num_epochs': 1671, 'hidden_size': 111, 'activation': 'relu', 'optimizer': 'SGD'}\nBest trial accuracy: 48.00%\n"